In [ ]:
import numpy as np

### 流量計算をするクラス  

#### フーリエの熱伝導  
定義：$\dot q = -\lambda\nabla・T$  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\lambda$：熱伝導率[W/mK]  
$T$：絶対温度[K]  

#### 熱伝達  
定義：$\dot q = -\alpha \Delta T$    
$\alpha$：熱伝達率[W/m2K]  

#### 熱伝導+相変化  
定義：$\dot q = -\lambda\nabla・T+ rW$  
$r$：相変化熱量[J/kg]  
$W$：水分の相変化量[kg/m3s]  

#### 気相水分流（水蒸気圧勾配・固体内）  
定義：$J_v = -\lambda^{'}_{p}\nabla・P_v$  
$J_v$：気相水分流量[kg/m2s]  
$\lambda^{'}_{p}$：水蒸気圧勾配に関する気相水分伝導率[kg/msPa]  
$P_v$：水蒸気圧[Pa]  

#### 気相水分流（水蒸気圧勾配・空気）  
定義：$J_v = -\alpha^{'}_{p}\Delta・P_v$  
$\alpha^{'}_{p}$：水分伝達率[kg/m2sPa]  

#### 気相水分流（水分化学ポテンシャル勾配・固体内）  
定義：$J_v = -\lambda^{'}_{\mu g}(\nabla・\mu+n_x g)-\lambda^{'}_{Tg}\nabla・T$  
$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に関する気相水分伝導率[kg/ms(J/kg)]  
$\lambda^{'}_{T g}  $：温度勾配に関する気相水分伝導率[kg/msK]  
$\mu$：水分化学ポテンシャル[J/kg]  
$n_x$：重力加速度に対する応答[-]（1なら垂直下向き、0なら水平方向）  
$g$：重力加速度  

#### 液相水分流（含水率勾配・固体内）  
定義：$J_l = -D_w\nabla・\phi$  
$J_l$：液相水分流量[kg/m2s]  
$D_w$：含水率勾配に関する液相水分伝導率[kg/m2s]  
$\phi$：含水率[Pa]  

#### 液相水分流（水分化学ポテンシャル勾配・固体内）  
定義：$J_l = -\lambda^{'}_{\mu l}(\nabla・\mu+n_x g)$  
$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に関する液相水分伝導率[kg/ms(J/kg)]  


In [3]:
class Flux():
    grav = 9.806650
    Mw = 18.02
    
    # 計算に必要な係数
    flist = {
        'フーリエ' : 'LAM',
        '熱伝達' : 'AL',
        '熱伝導+相変化熱' : 'LAM',
        
        '水蒸気圧勾配（固体内）': 'DP',
        '水蒸気圧勾配（空気）': 'ALDP',
        '水分化学ポテンシャル勾配（固体内）': ['LDMG','LDTG'],
        '水分化学ポテンシャル勾配（空気）': ['LDMG','LDTG'],
        
        '含水率勾配（固体内）': 'Dw',
        '水分化学ポテンシャル勾配（固体内）':'LDML',
        'なし': 'zero'
    }
    
################################################
    
    def __init__(self, moistureIndex, method ):
        self.method = method
        self.mainMI = moistureIndex
        self.prop = PropetyConversion()        
        
################################################
###     熱流     ###############################
    def Heat(transferCoefficient,Dtem,dx2,bcon):
###     フーリエ
        if self.method == "フーリエ":
            Lamda = self.prop.LAM(tem,moisture)
            return Lamda *Dtem/dx2
###     材料表面（熱伝達）
        elif self.method =="熱伝達":
            AL = self.prop.AL(tem,moisture)
            return AL *Dtem 
###     相変化熱
        elif self.method =="熱伝導+相変化熱":
            r = (597.5-0.559*(tem-273.16))*4186.05
            return r
        elif self.method == 'なし':
            return 0.0
        else : 
            raise ValueError('流量計算クラスのmethodには、flist内の言葉を指定しください。')

################################################
###     水蒸気流    ############################
    def Vapour(tem,moisture,moistureIndex,dx2,nx):
###     水蒸気圧勾配（固体内）
        if self.method == '水蒸気圧勾配（固体内）':
            LDP = self.prop.LDP(tem,moisture)
            return LDP *DPv/dx2
###     水蒸気圧勾配（空気）
        elif self.method == '水蒸気圧勾配（空気）':
            ALDP = self.prop.ALDP(tem,moisture)
            return ALDP *DPv

###     水分化学ポテンシャル勾配（固体内）
        elif self.method == '水分化学ポテンシャル勾配（固体内）':
            LDMG = self.prop.LDMG(tem,moisture)
            LDTG = self.prop.LDTG(tem,moisture)
            return LDMG *(Dmiu/dx2-nx*self.g) +LDTG*Dtem/dx2
###     ポテンシャル勾配（空気）
        elif self.method == '水分化学ポテンシャル勾配（空気）':
            LDMG = self.prop.LDMG(tem,moisture)
            LDTG = self.prop.LDTG(tem,moisture)
            return LDMG *(Dmiu/dx2-nx*self.g) +LDTG*Dtem/dx2 # ダミー
        elif self.method == 'なし':
            return 0.0
        else : 
            raise ValueError('流量計算クラスのmethodには、flist内の言葉を指定しください。')

#################################################
###     液相水分流    ###########################
    def Liquid(tem,moisture,moistureIndex,dx2,nx):
###     含水率勾配（固体内）
        if self.method == '含水率勾配（固体内）':
            Dw = self.prop.Dw(tem,moisture)
            return Dw *Dphi/dx2
###     水分化学ポテンシャル勾配（固体内）
        elif self.method == '水蒸気圧勾配（空気）':
            LDML = self.prop.LDML(tem,moisture)
            return LDML *(Dmiu/dx2-nx*self.g)
###     材料表面(液相水分は流れない)
        elif self.method == 'なし':
            return 0.0
        else : 
            raise ValueError('流量計算クラスのmethodには、flist内の言葉を指定しください。')

################################################